<a href="https://colab.research.google.com/github/SanjaySriram99/AgeDetectionProjectwithAzure/blob/main/Trainingmodel(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [ ]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')
from sklearn.preprocessing import OneHotEncoder,LabelEncoder # Label encoding, 1-hot encoding, multi-encoding
# LABEL binarizer is a 1-hot encoded MATRIX 
import cv2
import numpy as np
import random
import imutils
from imutils import paths
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [ ]:
!ls data

adolescent  college  middle_age  newborn  oldage  young_adult  young_child


In [ ]:
HP_dataset = 'data'
HP_model_path = 'bin/model'
HP_binarized_labels = 'bin/labels'
HP_metrics_storage = 'eval'
HP_test_dataset = 'test'
HP_epoch = 100
HP_init_lr = 5e-3 # learning_rate = 0.001
HP_batch_size = 16
HP_image_dim = (48,48,3)

In [ ]:
data = []
labels = [] 
# read all images
all_images = sorted(list(paths.list_images(HP_dataset)))

In [ ]:
import os
for impath in all_images:
  img = cv2.imread(impath)
  resized = cv2.resize(img, (HP_image_dim[0],HP_image_dim[1]) )
  imageData = img_to_array(resized)
  data.append(imageData)
  # extract label from filename (2nd last element) / \\ 
  label = impath.split(os.path.sep)[-2]
  labels.append(label)

In [ ]:
data = np.array(data, dtype='float' )/255.
labels = np.array(labels)
le = LabelEncoder()
labels1 = le.fit_transform(labels)

In [ ]:
from sklearn.model_selection import train_test_split
trainx,testx,trainy,testy = train_test_split(data, labels1, test_size=0.25, random_state=42)

In [ ]:
# Augmentation
aug = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, 
                         height_shift_range=0.2, shear_range=0.2,
                         zoom_range=0.2, horizontal_flip=True)

In [ ]:
from keras import backend
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.optimizers import Adam
#N= 5
HP_block1_conv_dim = 32
HP_block2_conv_dim = 64
HP_block3_conv_dim = 128
HP_block4_conv_dim = 256
HP_block5_dense_dim = 1024
HP_small_pattern = (3,3) # UNCOMPRESSED or 1-2 compression IMAGES
HP_large_pattern = (2,2) # 4 times compressed images from previous MP layers!!!
HP_dropout_little =0.20
HP_dropout_big = 0.50
# HP_epochs, batch_size-> are now problems of the developer USING this model. 

HP_img_dims = (48,48,3)

In [ ]:
from keras.applications.vgg16 import VGG16
from keras import layers, Model

base_model = VGG16(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

from keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final softmax layer for classification
x = layers.Dense(7, activation='softmax')(x)           

model = Model( base_model.input, x) 

model.summary()

In [ ]:
base_model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['acc'])

In [ ]:
hist = base_model.fit(aug.flow(trainx, trainy, batch_size=HP_batch_size),
                 validation_data=(testx, testy),
                 steps_per_epoch= len(trainx)//HP_batch_size,
                 epochs=HP_epoch, verbose=2)

In [ ]:
class RacoonVGG:
  @staticmethod
  def build(height, width, depth,classes):
    # assume that we are on TF, but if something else is detected, switch the dimension
    input_shape = (height, width, depth)
    channel_dim = -1 # last position 
    if backend.image_data_format() == 'channels_first':
      input_shape = (depth, height, width)
      channel_dim = 1
    model = Sequential()
    
    # BLOCK1
    model.add(Conv2D(HP_block1_conv_dim,HP_small_pattern, padding='same',
                     input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_small_pattern))
    model.add(Dropout(HP_dropout_little))


    # COMPLEX BLOCK 2
    model.add(Conv2D(HP_block2_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(HP_block2_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_large_pattern))
    model.add(Dropout(HP_dropout_little))
    
    # COMPLEX BLOCK 3
    model.add(Conv2D(HP_block3_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(HP_block3_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_large_pattern))
    model.add(Dropout(HP_dropout_little))

    # COMPLEX BLOCK 4
    model.add(Conv2D(HP_block4_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(HP_block4_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_large_pattern))
    model.add(Dropout(HP_dropout_little))

    # BLOCK 5- Image Classification (OBJECT)
    model.add(Flatten())
    model.add(Dense(HP_block5_dense_dim))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(HP_dropout_big))
    model.add(Dense(classes))
    model.add(Activation('softmax'))

    return model    

In [ ]:
# from sklearn.model_selection import GridSearchCV

model = RacoonVGG.build(height=HP_img_dims[0], width=HP_img_dims[1], depth=HP_img_dims[2],classes = len(le.classes_))
opt = Adam(lr=HP_init_lr, decay = HP_init_lr/ HP_epoch)
model.compile(loss= 'sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
hist = model.fit(aug.flow(trainx, trainy, batch_size=HP_batch_size),
                 validation_data=(testx, testy),
                 steps_per_epoch= len(trainx)//HP_batch_size,
                 epochs=HP_epoch, verbose=1)


Epoch 1/100
73/73 [==============================] - 19s 261ms/step - loss: 3.3427 - accuracy: 0.1667 - val_loss: 74.9965 - val_accuracy: 0.1270
Epoch 2/100
73/73 [==============================] - 18s 248ms/step - loss: 3.2319 - accuracy: 0.1883 - val_loss: 13.6799 - val_accuracy: 0.1329
Epoch 3/100
73/73 [==============================] - 18s 249ms/step - loss: 2.9226 - accuracy: 0.1813 - val_loss: 4.2092 - val_accuracy: 0.1806
Epoch 4/100
73/73 [==============================] - 18s 248ms/step - loss: 2.9454 - accuracy: 0.1770 - val_loss: 6.8294 - val_accuracy: 0.1468
Epoch 5/100
73/73 [==============================] - 18s 248ms/step - loss: 3.0609 - accuracy: 0.1727 - val_loss: 10.9939 - val_accuracy: 0.1349
Epoch 6/100
73/73 [==============================] - 18s 249ms/step - loss: 2.8489 - accuracy: 0.1693 - val_loss: 2.9512 - val_accuracy: 0.1468
Epoch 7/100
73/73 [==============================] - 18s 248ms/step - loss: 2.9765 - accuracy: 0.1744 - val_loss: 2.9126 - val_accur

In [ ]:
from keras.applications.inception_v3 import InceptionV3

In [ ]:
pre_trained_model = InceptionV3(input_shape = (224, 224, 3), # Shape of our images
                                include_top = False, # Leave out the last fully connected layer
                                weights = 'imagenet')

In [ ]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [ ]:
from keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(pre_trained_model.output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)           

model = Model( pre_trained_model.input, x) 

pre_trained_model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['acc'])

AttributeError: ignored

In [ ]:
hist = pre_trained_model.fit(aug.flow(trainx, trainy, batch_size=HP_batch_size),
                 validation_data=(testx, testy),
                 steps_per_epoch= len(trainx)//HP_batch_size,   #// batch_size for larger datasets
                 epochs=HP_epoch, verbose=1)

ValueError: ignored